# Test of Exponential Diode: VS-R-Diode in series

## Setup Simulation

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy
import re
import os

#%matplotlib widget

#Define simulation scenario
time_step = 0.0001
final_time = 0.1 #0.006
simName = "EMT_Ph1_VS_R1_ExponentialDiode"

dpsimpy.Logger.set_log_dir('logs/' + simName)

#Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.Single)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.Single)

#Components
vs = dpsimpy.emt.ph1.VoltageSource("vs")
vs.set_parameters(complex(1.,0.), 50.0);

load = dpsimpy.emt.ph1.Resistor("r1")
load.set_parameters(10.)

expDiode = dpsimpy.emt.ph1.ExponentialDiode("ExponentialDiode");

#Topology
vs.connect([gnd, n1]);
load.connect([n2, n1]);
expDiode.connect([gnd, n2]);

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, load, expDiode])

#Logging
logger = dpsimpy.Logger(simName)
logger.log_attribute("I_ExponentialDiode", "i_intf", expDiode)
logger.log_attribute("V_ExponentialDiode", "v_intf", expDiode)
logger.log_attribute("V_Resistor", "v_intf", load)
logger.log_attribute("V_VS", "v_intf", vs)

# Simulation
sim = dpsimpy.Simulation(simName)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.ITERATIVEMNA)
sim.add_logger(logger)
sim.run()

## Read and visualize results

In [ ]:
path_logfile = os.getcwd() + "/logs/" + simName + '/' + simName + '.csv'
ts_dpsim_EMT = read_timeseries_dpsim(path_logfile)

plt.close('all')
fig1 = plt.figure()

plt.plot(ts_dpsim_EMT['V_ExponentialDiode'].time, ts_dpsim_EMT['V_ExponentialDiode'].values, "b-", label='V_D')
plt.plot(ts_dpsim_EMT['V_Resistor'].time, ts_dpsim_EMT['V_Resistor'].values, "g-", label='V_R')

plt.legend(loc = 4)

plt.title('Voltages')
plt.xlabel('t [s]')
plt.ylabel('Voltage [V]')

fig2 = plt.figure()

plt.plot(ts_dpsim_EMT['I_ExponentialDiode'].time, ts_dpsim_EMT['I_ExponentialDiode'].values, "r-", label='I_D')

plt.legend(loc = 4)

plt.title('ExponentialDiode Current')
plt.xlabel('t [s]')
plt.ylabel('Current [A]')